# Currency data API's

## Live Data extraction

Lambda layer: group2_currency_api

In [ ]:
import json
import requests
import psycopg2
import boto3
import os
from datetime import datetime

In [ ]:
# Load sensitive configuration values from environment variables.
# This keeps credentials and connection details out of the codebase.
ENDPOINT = os.environ['ENDPOINT']
DB_NAME = os.environ['DB_NAME']
USERNAME = os.environ['USERNAME']
PASSWORD = os.environ['PASSWORD']
S3_BUCKET = os.environ['S3_BUCKET'] 
API_KEY = os.environ['API_KEY']

In [ ]:
def fetch_currency_data(source_currency="CHF", target_currencies="EUR"):
    """Fetch live exchange rates with customizable source and target currencies"""

    # Ensure API key is available
    if not API_KEY:
        return {"error": "API key not found"}

    # Build API request URL
    base_url = "https://api.apilayer.com/currency_data/"
    endpoint = "live"
    api_url = f"{base_url}{endpoint}?source={source_currency}&currencies={target_currencies}"
    headers = {"apikey": API_KEY}

    # Send request to currency API & return parsed JSON if successful, otherwise return error message
    response = requests.get(api_url, headers=headers)
    return response.json() if response.status_code == 200 else {"error": response.text}

def store_currency_data_in_rds(currency_data):
    """Store live exchange rates in an AWS RDS PostgreSQL database."""
    try:
        print("Connecting to DB...")

        # Establish DB connection using psycopg2
        conn = psycopg2.connect(
            host=ENDPOINT,
            dbname=DB_NAME,
            user=USERNAME,
            password=PASSWORD
        )
        cur = conn.cursor()
        conn.set_session(autocommit=True)
        print("DB connection successful.")

        # Define the table schema and columns
        table_name = "tbl_currency_data"
        columns = {
            "timestamp": "TIMESTAMP NOT NULL",
            "source_currency": "VARCHAR(10)",
            "target_currency": "VARCHAR(10)",
            "exchange_rate": "FLOAT"
        }

        # Create table dynamically if it doesn't exist
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
        create_table_query += ", ".join([f"{col} {datatype}" for col, datatype in columns.items()])
        create_table_query += ", PRIMARY KEY (timestamp, source_currency, target_currency));"
        cur.execute(create_table_query)

        # Prepare INSERT statement with conflict handling
        insert_query = f"""
        INSERT INTO {table_name} ({', '.join(columns.keys())})
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (timestamp, source_currency, target_currency) DO NOTHING;
        """

        # Get current timestamp in milliseconds (trimmed to 3 decimals)
        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]  

        # Parse currency pairs and insert them into the DB
        for pair, rate in currency_data.get("quotes", {}).items():
            source_currency, target_currency = pair[:3], pair[3:]
            cur.execute(
                insert_query,
                (now, source_currency, target_currency, rate)
            )

        # Clean up DB connection
        cur.close()
        conn.close()

    except psycopg2.Error as e:
        print("Database error:", e)

def convert_unix_to_iso(timestamp):
    """Convert Unix timestamp to ISO 8601 format"""
    return datetime.utcfromtimestamp(timestamp).isoformat()

def preprocess_currency_data(data):
    """Fix timestamps in the currency data JSON"""
    if "timestamp" in data:
        data["timestamp"] = convert_unix_to_iso(data["timestamp"])
    return data

def store_json_to_s3(currency_data):
    """Store the JSON data to S3 bucket in a structured way"""
    # Initialize S3 client
    s3 = boto3.client('s3')

    # Generate file key based on today's date
    today = str(datetime.today().date())
    foldername = "currency_data"
    key = f"{foldername}/{foldername}_{today}.json"

    # Convert timestamps to ISO format
    processed_data = preprocess_currency_data(currency_data)

    # Upload JSON to S3
    s3.put_object(
        Bucket=S3_BUCKET,
        Key=key,
        Body=json.dumps(processed_data, indent=2),
        ContentType='application/json'
    )

def lambda_handler(event, context):
    """AWS Lambda Entry Point"""
    
    # Step 1: Fetch exchange rate data from external API
    currency_data = fetch_currency_data()
    
    # Step 2: Return early if fetch failed
    if "error" in currency_data:
        return {
            "statusCode": 400,
            "body": json.dumps({"error": currency_data["error"]}, indent=2)
        }
    
    # Step 3: Store data in PostgreSQL RDS
    store_currency_data_in_rds(currency_data)

    # Step 4: Store JSON data in S3
    store_json_to_s3(currency_data)

    # Step 5: Return success response
    return {
        "statusCode": 200,
        "body": json.dumps({
            "message": "Live exchange rates stored successfully in RDS and S3.",
            "s3_path": f"s3://{S3_BUCKET}/currency_data/currency_rates_{datetime.today().strftime('%Y-%m-%d')}.json",
            "currency_rates": currency_data
        }, indent=2)
    }

## Historical data extraction:

Lambda Layer: group2_historical_data

In [ ]:
# ===================================================================================
# AWS Lambda Function: Historical Currency Data Pipeline
# Description: Fetches historical exchange rates from an external API and stores
#              the data in an RDS PostgreSQL database and in an S3 bucket as JSON.
# Technologies: AWS Lambda, S3, RDS (PostgreSQL), apilayer API
# Author: Joshua Hügli
# ===================================================================================

import json
import requests
import psycopg2
import boto3
import os
from datetime import datetime

# ------------------------------------------------------------------------------
# Environment Configuration
# Load sensitive configuration values from Lambda environment variables.
# ------------------------------------------------------------------------------

ENDPOINT = os.environ['ENDPOINT']       # RDS database host
DB_NAME = os.environ['DB_NAME']         # Database name
USERNAME = os.environ['USERNAME']       # Database username
PASSWORD = os.environ['PASSWORD']       # Database password
S3_BUCKET = os.environ['S3_BUCKET']     # S3 bucket name for JSON export
API_KEY = os.environ['API_KEY']         # API key for currency data provider


# ------------------------------------------------------------------------------
# Fetch Historical Currency Data
# ------------------------------------------------------------------------------

def fetch_historical_currency_data(start_date="2024-01-01", end_date="2025-04-29", source_currency="CHF", target_currencies="EUR"):
    """
    Fetch historical exchange rate data from the apilayer API.

    Args:
        start_date (str): Start date in YYYY-MM-DD format.
        end_date (str): End date in YYYY-MM-DD format.
        source_currency (str): The base currency (default: CHF).
        target_currencies (str): Comma-separated target currencies (default: EUR).

    Returns:
        dict: JSON response with historical exchange rates or error message.
    """
    if not API_KEY:
        return {"error": "API key not found"}

    base_url = "https://api.apilayer.com/currency_data/timeframe"
    params = f"?start_date={start_date}&end_date={end_date}&source={source_currency}&currencies={target_currencies}"
    url = base_url + params
    headers = {"apikey": API_KEY}

    response = requests.get(url, headers=headers)
    return response.json() if response.status_code == 200 else {"error": response.text}


# ------------------------------------------------------------------------------
# Store Historical Currency Data in RDS
# ------------------------------------------------------------------------------

def store_historical_currency_data_in_rds(currency_data):
    """
    Store historical exchange rates in a PostgreSQL RDS instance.

    Args:
        currency_data (dict): JSON object with exchange rates from API.
    """
    try:
        print("Connecting to DB...")
        conn = psycopg2.connect(
            host=ENDPOINT,
            dbname=DB_NAME,
            user=USERNAME,
            password=PASSWORD
        )
        cur = conn.cursor()
        conn.set_session(autocommit=True)
        print("DB connection successful.")

        # Define schema
        table_name = "tbl_currency_data"
        columns = {
            "timestamp": "TIMESTAMP NOT NULL",
            "source_currency": "VARCHAR(10)",
            "target_currency": "VARCHAR(10)",
            "exchange_rate": "FLOAT"
        }

        # Create table if not exists
        create_table_query = f"CREATE TABLE IF NOT EXISTS {table_name} ("
        create_table_query += ", ".join([f"{col} {datatype}" for col, datatype in columns.items()])
        create_table_query += ", PRIMARY KEY (timestamp, source_currency, target_currency));"
        cur.execute(create_table_query)

        # Prepare INSERT statement
        insert_query = f"""
        INSERT INTO {table_name} ({', '.join(columns.keys())})
        VALUES (%s, %s, %s, %s)
        ON CONFLICT (timestamp, source_currency, target_currency) DO NOTHING;
        """

        # Insert all exchange rate records
        for date_str, rates in currency_data.get("quotes", {}).items():
            for pair, rate in rates.items():
                source_currency, target_currency = pair[:3], pair[3:]
                timestamp = datetime.strptime(date_str, "%Y-%m-%d")
                cur.execute(insert_query, (timestamp, source_currency, target_currency, rate))

        cur.close()
        conn.close()

    except psycopg2.Error as e:
        print("Database error:", e)


# ------------------------------------------------------------------------------
# Store JSON Data in S3
# ------------------------------------------------------------------------------

def store_json_to_s3(currency_data, start_date, end_date):
    """
    Store the fetched historical currency data as a JSON file in an S3 bucket.

    Args:
        currency_data (dict): JSON data from the API.
        start_date (str): Start date of the data range.
        end_date (str): End date of the data range.
    """
    s3 = boto3.client('s3')
    foldername = "currency_data"
    key = f"{foldername}/currency_data_{start_date}_to_{end_date}.json"

    s3.put_object(
        Bucket=S3_BUCKET,
        Key=key,
        Body=json.dumps(currency_data, indent=2),
        ContentType='application/json'
    )


# ------------------------------------------------------------------------------
# AWS Lambda Entry Point
# ------------------------------------------------------------------------------

def lambda_handler(event, context):
    """
    Lambda function entry point for fetching and storing historical currency data.

    Args:
        event (dict): Event payload with optional 'start_date' and 'end_date'.
        context (LambdaContext): Runtime information.

    Returns:
        dict: HTTP-style status and message with S3 path reference.
    """
    # Default values if not provided in event
    start_date = event.get("start_date", "2025-05-22")
    end_date = event.get("end_date", "2025-05-23")

    currency_data = fetch_historical_currency_data(start_date=start_date, end_date=end_date)

    if "error" in currency_data:
        return {
            "statusCode": 400,
            "body": json.dumps({"error": currency_data["error"]}, indent=2)
        }

    store_historical_currency_data_in_rds(currency_data)
    store_json_to_s3(currency_data, start_date, end_date)

    return {
        "statusCode": 200,
        "body": json.dumps({
            "message": "Historical exchange rates stored successfully in RDS and S3.",
            "s3_path": f"s3://{S3_BUCKET}/currency_data/currency_data_{start_date}_to_{end_date}.json",
            "currency_rates": currency_data
        }, indent=2)
    }
